In [48]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Modeling") \
    .getOrCreate()

In [49]:
# Get the final data for modeling
file_path = '../data/final_data_for_modeling.parquet'
final_sdf = spark.read.parquet(file_path)
final_sdf.show(5, vertical = True)

-RECORD 0--------------------------------------
 PULocationID           | 257                  
 date                   | 2024-05-04           
 hour_bucket            | 12                   
 market_share_yellow    | 0.6134969325153374   
 temp                   | 0.5845323741007195   
 rhum                   | 0.574898785425101    
 prcp                   | 0.0                  
 walkup_apartment       | 0.38260763437633494  
 elevator_apartment     | 0.1541458552776615   
 garage_n_gas           | 0.08240738669733964  
 hotel                  | 0.0                  
 hospital               | 0.0                  
 theatre                | 0.0                  
 store                  | 0.07730168860951439  
 church                 | 0.31566503937379364  
 asylum                 | 0.006835418499575569 
 office                 | 0.003478216697113... 
 public_n_cultural      | 0.03875960932793963  
 condo                  | 0.055875674247014936 
 multiple_use_residence | 0.397044928065

# Lasso Regression model

In [50]:
# Split the data into training and test sets based on the month
train_sdf_lasso = final_sdf.filter(final_sdf["month"] != 5)  
test_sdf_lasso = final_sdf.filter(final_sdf["month"] == 5)   # Test on data from may

In [51]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# One-Hot Encoding for categorical features
indexers = [StringIndexer(inputCol=col_name, outputCol=col_name + "_index").fit(final_sdf) 
            for col_name in ["PULocationID", "hour_bucket", "day_of_week"]]

encoders = [OneHotEncoder(inputCol=col_name + "_index", outputCol=col_name + "_onehot") 
            for col_name in ["PULocationID", "hour_bucket", "day_of_week"]]

# List of features (keeping PULocationID as is for grouping)
feature_cols = ["temp", "rhum", "prcp", "avg_land_value"] + \
               ["walkup_apartment", "elevator_apartment", "garage_n_gas", "hotel", "hospital",
                "theatre", "store", "church", "asylum", "office", "public_n_cultural", "condo",
                "multiple_use_residence", "transport", "utility", "vacant_land", "education",
                "government", "miscellanous", "family_dwelling", "industrial"] + \
               ["PULocationID_onehot", "hour_bucket_onehot", "day_of_week_onehot"]

# Combine all features into a single vector column
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Build a pipeline for indexing, encoding, assembling features
pipeline = Pipeline(stages=indexers + encoders + [assembler])

# Fit the pipeline on the training data
train_sdf_lasso = pipeline.fit(train_sdf_lasso).transform(train_sdf_lasso)
test_sdf_lasso = pipeline.fit(test_sdf_lasso).transform(test_sdf_lasso)

# Keep only the necessary columns for modeling and analysis
train_sdf_lasso = train_sdf_lasso.select("PULocationID", "features", "market_share_yellow")
test_sdf_lasso = test_sdf_lasso.select("PULocationID", "features", "market_share_yellow")

# Train the Lasso model
lasso = LinearRegression(featuresCol="features", labelCol="market_share_yellow", elasticNetParam=1.0, regParam=0.02)
lasso_model = lasso.fit(train_sdf_lasso)

# Get the coefficients and corresponding feature names
coefficients = lasso_model.coefficients
intercept = lasso_model.intercept

# Map coefficients back to feature names
non_zero_coefficients = [(feature, coeff) for feature, coeff in zip(feature_cols, coefficients) if coeff != 0]

# Print out the non-zero coefficients
print("Non-zero coefficients:")
for feature, coeff in non_zero_coefficients:
    print(f"{feature}: {coeff}")

# Make predictions on the test data
predictions_lasso = lasso_model.transform(test_sdf_lasso)

# Show the predictions
predictions_lasso.select("features", "market_share_yellow", "prediction").show(5)

# Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="market_share_yellow", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_lasso)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# evaluate with other metrics like R-squared
r2 = evaluator.evaluate(predictions_lasso, {evaluator.metricName: "r2"})
print(f"R-squared: {r2}")

Non-zero coefficients:
temp: 0.8847975514623099
rhum: -0.3790170347340133
prcp: -0.6598152673674832
avg_land_value: 35.18929761911898
walkup_apartment: -0.4501453550609017
garage_n_gas: -1.301420377743595
hotel: 9.922884947314833
store: -0.43520079384123606
church: -0.25067715843752597
public_n_cultural: 0.31993269441904776
multiple_use_residence: -0.4004804744619177
transport: 13.429106182912735
education: -0.1041299722791314
miscellanous: 0.022608452155879172
hour_bucket_onehot: 2.6528103910127836
+--------------------+-------------------+------------------+
|            features|market_share_yellow|        prediction|
+--------------------+-------------------+------------------+
|(295,[0,1,3,4,5,6...| 0.6134969325153374|2.0553982834497813|
|(295,[0,1,3,4,5,6...|0.15420200462606012| 2.189965618149156|
|(295,[0,1,3,4,5,6...|  45.30020703933747|38.827881945193724|
|(295,[0,1,3,4,5,6...|  34.16815742397138|24.872241182481122|
|(295,[0,1,3,4,5,6...|                0.0|2.0225649794456224|

# Checking for zero coefficent 

In [52]:
# Print out the coefficients with zero values
zero_coefficients = [(feature, coeff) for feature, coeff in zip(feature_cols, coefficients) if coeff == 0]

print("Zero coefficients:")
for feature, coeff in zero_coefficients:
    print(f"{feature}: {coeff}")

Zero coefficients:
elevator_apartment: 0.0
hospital: 0.0
theatre: 0.0
asylum: 0.0
office: 0.0
condo: 0.0
utility: 0.0
vacant_land: 0.0
government: 0.0
family_dwelling: 0.0
industrial: 0.0
PULocationID_onehot: 0.0
day_of_week_onehot: 0.0


In [53]:
# Indexing the categorical features
indexers = [StringIndexer(inputCol=col_name, outputCol=col_name + "_index").fit(final_sdf) 
            for col_name in ["PULocationID", "hour_bucket", "day_of_week", "month"]]

# Combine indexed features with numerical features
feature_cols = ["temp", "rhum", "prcp", "avg_land_value"] + \
               ["walkup_apartment", "elevator_apartment", "garage_n_gas", "hotel", "hospital",
                "theatre", "store", "church", "asylum", "office", "public_n_cultural", "condo",
                "multiple_use_residence", "transport", "utility", "vacant_land", "education",
                "government", "miscellanous", "family_dwelling", "industrial"] + \
               ["PULocationID_index", "hour_bucket_index", "day_of_week_index", "month_index"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Build the pipeline for indexing and assembling features
pipeline = Pipeline(stages=indexers + [assembler])

# Fit the pipeline on the entire data
final_sdf = pipeline.fit(final_sdf).transform(final_sdf)


In [54]:
# Split the data into training and test sets based on the month (May is the test set)
train_sdf_forest = final_sdf.filter(final_sdf["month_index"] != 4)  
test_sdf_forest = final_sdf.filter(final_sdf["month_index"] == 4)

# Keep only the features and the target variable for modeling
train_sdf_forest = train_sdf_forest.select("PULocationID", "features", "market_share_yellow")
test_sdf_forest = test_sdf_forest.select("PULocationID", "features", "market_share_yellow")

# Increase executor memory

In [55]:
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.executor.memory", "6g") 

# Random Forest Model

In [56]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Sample the training data for hyperparameter tuning
train_sdf_reduced = train_sdf_forest.sample(fraction=0.1)

# Initialize the RandomForestRegressor
rf = RandomForestRegressor(featuresCol="features", labelCol="market_share_yellow")

# Create a parameter grid for hyperparameter tuning
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 7]) \
    .addGrid(rf.maxBins, [260, 370, 380]) \
    .build()

# Initialize the RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="market_share_yellow", predictionCol="prediction", metricName="rmse")

# Set up cross-validation
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

# Fit the Random Forest model with cross-validation on the reduced training data
cvModel = cv.fit(train_sdf_reduced)

# Extract the best hyperparameters from cross-validation
best_num_trees = cvModel.bestModel.getNumTrees
best_max_depth = cvModel.bestModel.getMaxDepth()
best_max_bins = cvModel.bestModel.getMaxBins()

print(f"Best Number of Trees: {best_num_trees}")
print(f"Best Max Depth: {best_max_depth}")
print(f"Best Max Bins: {best_max_bins}")

# Train the final model on the full training data using the best hyperparameters
final_rf = RandomForestRegressor(
    featuresCol="features", 
    labelCol="market_share_yellow",
    numTrees=best_num_trees,
    maxDepth=best_max_depth,
    maxBins=best_max_bins
)

final_model = final_rf.fit(train_sdf_forest)

# Make predictions on the test data
predictions_rf = final_model.transform(test_sdf_forest)

# Evaluate the model
rmse = evaluator.evaluate(predictions_rf)
r2 = evaluator.evaluate(predictions_rf, {evaluator.metricName: "r2"})

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")

Best Number of Trees: 20
Best Max Depth: 7
Best Max Bins: 380


Root Mean Squared Error (RMSE): 2.9984873443545186
R-squared: 0.9301261010409843


# Analyze random forest's feature of importance

In [57]:
# Get the feature importances from the trained Random Forest model
importances = final_model.featureImportances

# Zip the feature importances with their corresponding feature names
feature_importance = [(feature, importance) for feature, importance in zip(feature_cols[1:], importances)]

# Sort the features by importance in descending order
feature_importance_sorted = sorted(feature_importance, key=lambda x: x[1], reverse=True)

# Print out the most important features
print("Feature Importances:")
for feature, importance in feature_importance_sorted:
    print(f"{feature}: {importance:.4f}")

Feature Importances:
hour_bucket_index: 0.4671
walkup_apartment: 0.1874
multiple_use_residence: 0.0853
hospital: 0.0831
day_of_week_index: 0.0665
education: 0.0304
hotel: 0.0153
industrial: 0.0116
garage_n_gas: 0.0103
vacant_land: 0.0045
family_dwelling: 0.0042
utility: 0.0036
transport: 0.0033
church: 0.0033
miscellanous: 0.0031
asylum: 0.0031
public_n_cultural: 0.0029
elevator_apartment: 0.0025
store: 0.0025
condo: 0.0021
theatre: 0.0016
office: 0.0013
PULocationID_index: 0.0013
month_index: 0.0013
government: 0.0012
rhum: 0.0004
prcp: 0.0003
avg_land_value: 0.0001


# Calculate average error per location of each models

In [58]:
from pyspark.sql.functions import col, expr

# Calculate the error
predictions_rf = predictions_rf.withColumn("error", col("prediction") - col("market_share_yellow"))

In [59]:
# Get mean errors for each location
mean_errors = predictions_rf.groupBy("PULocationID").agg(expr("mean(error)").alias("mean_error"))

In [60]:
import geopandas as gpd
import pandas as pd

# Load the taxi zones shapefile
sf = gpd.read_file("../data/raw/taxi_zones/taxi_zones.shp")

# Reproject the geometry to WGS84 (Longitude/Latitude)
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# Convert mean_errors to Pandas DataFrame
mean_errors_df = mean_errors.toPandas()

# Merge with GeoDataFrame containing geographical data
gdf = gpd.GeoDataFrame(pd.merge(mean_errors_df, sf, left_on='PULocationID', right_on='LocationID')) \
                        .drop('LocationID', axis=1)

In [61]:
# Calculate the error (difference between prediction and actual market share)
predictions_lasso = predictions_lasso.withColumn("error", expr("prediction - market_share_yellow"))

In [62]:
# Group by PULocationID and calculate mean error
mean_errors_lasso = predictions_lasso.groupBy("PULocationID").agg(expr("mean(error)").alias("mean_error"))


# Convert to Pandas DataFrame
mean_errors_lasso_df = mean_errors_lasso.toPandas()

In [63]:
# Merge the mean_errors_lasso DataFrame with the GeoDataFrame
gdf_lasso = gpd.GeoDataFrame(pd.merge(mean_errors_lasso_df, sf, left_on='PULocationID', right_on='LocationID')) \
                        .drop('LocationID', axis=1)

# Visualizing for Lasso regression average error distribution per location

In [64]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from folium import Map, TileLayer, Choropleth, LayerControl
import numpy as np

# Convert the merged GeoDataFrame to GeoJSON format for Folium
geoJSON_lasso = gdf_lasso[['PULocationID', 'geometry']].to_json()

# Create the base map
m_lasso = Map(location=[40.66, -73.94], zoom_start=10, tiles=None)
TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m_lasso)

# Create a custom color map that goes from blue (negative) to white (zero) to red (positive)
colors = [(0, "blue"), (0.5, "white"), (1, "red")]
custom_cmap = LinearSegmentedColormap.from_list("RdWhBu", colors)

# Calculate the max absolute error to make the color scale symmetric around zero
max_error = max(abs(gdf_lasso['mean_error'].max()), abs(gdf_lasso['mean_error'].min()))

# Set normalization to ensure the color map is symmetric around zero
norm = Normalize(vmin=-max_error, vmax=max_error)

# Convert color values into a list of RGBA values
gdf_lasso['color'] = gdf_lasso['mean_error'].apply(lambda x: plt.cm.ScalarMappable(norm=norm, cmap=custom_cmap).to_rgba(x))

# Create more bins to capture finer differences
bins = np.linspace(-max_error, max_error, num=200)

# Create the Choropleth map
Choropleth(
    geo_data=geoJSON_lasso,
    name="choropleth",
    data=gdf_lasso,
    columns=["PULocationID", "mean_error"],
    key_on="properties.PULocationID",
    fill_color='RdYlBu',  # Using color brewer palette, closest to your custom cmap
    fill_opacity=0.9,
    line_opacity=0.4,
    legend_name="Average Prediction Error",
    bins=bins,
    nan_fill_color="white",
    nan_fill_opacity=0.6
).add_to(m_lasso)

# Add layer control and save the map
LayerControl().add_to(m_lasso)
fpth_lasso = "../plots/lasso_error_map.html"
m_lasso.save(fpth_lasso)

print(f"Map saved to: {fpth_lasso}")

Map saved to: ../plots/lasso_error_map.html


# Visualizing for Random forest average error distribution per location

In [65]:
# Convert the merged GeoDataFrame to GeoJSON format for Folium
geoJSON_rf = gdf[['PULocationID', 'geometry']].to_json()

# Create the base map for Random Forest
m_rf = Map(location=[40.66, -73.94], zoom_start=10, tiles=None)
TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m_rf)

# Set normalization with balanced extremes for Random Forest
max_abs_error_rf = max(abs(gdf['mean_error'].min()), abs(gdf['mean_error'].max()))
norm_rf = Normalize(vmin=-max_abs_error_rf, vmax=max_abs_error_rf)

# Convert color values into a list of RGBA values for Random Forest
gdf['color'] = gdf['mean_error'].apply(lambda x: plt.cm.ScalarMappable(norm=norm_rf, cmap=custom_cmap).to_rgba(x))

# Create more bins to capture finer differences 
bins_rf = np.linspace(-max_abs_error_rf, max_abs_error_rf, num=200)

# Create the Choropleth map for Random Forest
Choropleth(
    geo_data=geoJSON_rf,
    name="choropleth",
    data=gdf,
    columns=["PULocationID", "mean_error"],
    key_on="properties.PULocationID",
    fill_color='RdYlBu',
    fill_opacity=0.9,
    line_opacity=0.4,
    legend_name="Average Prediction Error",
    bins=bins_rf,
    nan_fill_color="white",
    nan_fill_opacity=0.6
).add_to(m_rf)

# Add layer control and save the map for Random Forest
LayerControl().add_to(m_rf)
fpth_rf = "../plots/mean_error_map_random_forest.html"
m_rf.save(fpth_rf)

print(f"Map saved to: {fpth_rf}")

Map saved to: ../plots/mean_error_map_random_forest.html
